In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('combined-dataset/combined_datasetV2.csv')
df.dropna(subset=['rating', 'review 1', 'review 2', 'review 3', 'review 4', 'review 5'], inplace=True)

In [ ]:
filtered = df[df['rating-count'] >= 50]
filtered.reset_index(drop=True, inplace=True)
filtered = filtered[filtered['rating'] >= 3.5]
filtered

In [ ]:
review_columns = ['review 1', 'review 2', 'review 3', 'review 4', 'review 5']
reviews_df = filtered.melt(id_vars=['id', 'types'], value_vars=review_columns, var_name='review_number', value_name='review')
reviews_df.dropna(subset=['review'], inplace=True)
reviews_df.reset_index(drop=True, inplace=True)

In [ ]:
reviews_df

In [ ]:
reviews_df.to_csv('combined-dataset/reviews_data.csv', index=False)

In [ ]:
import random
total_reviews = len(reviews_df)

In [ ]:
unique_ratio = 0.5
num_unique_users = int(total_reviews * unique_ratio)
total_users = int(total_reviews / unique_ratio)

In [ ]:
users_id = [f'user_{i + 1}' for i in range(total_users)]

In [ ]:
reviews_df['user_id'] = [random.choice(users_id) for _ in range(total_reviews)]

In [ ]:
user_review_counts = reviews_df['user_id'].value_counts()
print(user_review_counts.describe())

In [ ]:
reviews_df

In [ ]:
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
reviews_df['sentiment'] = reviews_df['review'].apply(lambda review: TextBlob(review).sentiment.polarity)
reviews_df['sentiment'] = MinMaxScaler().fit_transform(reviews_df[['sentiment']])

In [ ]:
reviews_df

In [ ]:
reviews_df['sentiment'] = ((reviews_df['sentiment'] - 0) / (1 - 0)) * (5 - 1) + 1

In [ ]:
reviews_df

In [ ]:
reviews_df.to_csv('combined-dataset/final_reviews_data.csv', index=False)

In [ ]:
import pandas as pd
import random
from textblob import TextBlob
import matplotlib.pyplot as plt

# Load the reviews data
reviews_df = pd.read_csv('combined-dataset/reviews_data.csv')

# Calculate total number of reviews
total_reviews = len(reviews_df)

# Desired unique-to-multiple review ratio
unique_ratio = 0.70

# Number of unique users
num_unique_users = int(total_reviews * unique_ratio)

# Number of total users, including those with multiple reviews
total_users = int(total_reviews / unique_ratio)  # Rough estimate

# Generate synthetic user IDs
user_ids = [f"user_{i+1}" for i in range(total_users)]

# Determine reviews per user randomly
min_reviews_per_user = 1
max_reviews_per_user = 5  # Adjust as needed

# Create a mapping of user_id to the number of reviews they will have
user_review_map = {user_id: random.randint(min_reviews_per_user, max_reviews_per_user) for user_id in user_ids}

# Flatten the user review assignment
assigned_users = []
for user_id, count in user_review_map.items():
    assigned_users.extend([user_id] * count)

# If there are more reviews than assigned users, extend the list randomly
if len(assigned_users) < total_reviews:
    extra_users = [random.choice(user_ids) for _ in range(total_reviews - len(assigned_users))]
    assigned_users.extend(extra_users)

# Shuffle to ensure randomness
random.shuffle(assigned_users)

# Assign the shuffled list to the DataFrame
reviews_df['user_id'] = assigned_users[:total_reviews]

# Check the distribution of reviews per user
user_review_counts = reviews_df['user_id'].value_counts()
print(user_review_counts.describe())

# Perform sentiment analysis
reviews_df['sentiment'] = reviews_df['review'].apply(lambda review: TextBlob(review).sentiment.polarity)

# Visualize sentiment distribution
plt.hist(reviews_df['sentiment'], bins=50, edgecolor='k', alpha=0.7)
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment Score')
plt.ylabel('Frequency')
plt.show()

# Save the final DataFrame
reviews_df.to_csv('combined-dataset/final_reviews_data.csv', index=False)


In [ ]:
reviews_df

In [ ]:
# get unique primary types